# Plot various heatmaps of the integrated data  
[**Heatmap with metadata profile**](#meta)  
[**Heatmap with correlation matrix**](#corr)  
[**Heatmap with deviation from mean**](#mean)  

In [ ]:
%matplotlib widget
import DanMAX as DM
import numpy as np
import matplotlib.pyplot as plt
import h5py
style = DM.darkMode(style_dic={'figure.figsize':'large'})

In [ ]:
fname = DM.findScan()

aname = DM.getAzintFname(fname)

data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    Q = True
else:
    x = data['tth']
    Q = False
    
meta = DM.getMetaData(fname)

# reduce time-resolution to speed up initial analysis
rf = 1
start = None 
end =  None
data = DM.reduceDic(data,reduction_factor=rf,start=start,end=-1)
meta = DM.reduceDic(meta,reduction_factor=rf,start=start,end=end)

I = data['I']
t = meta['time']
T = meta['temp']
I0 = meta['I0']

# normalize
I = (I.T/I0).T

print(f'Effective time-resolution: {np.mean(np.diff(t)):.2f} s')

## Plot heatmap with meta data <a id='meta'></a>

In [ ]:
# initialize figure
fig = plt.figure()
fig.tight_layout()

# initialize grid and subplot with different size-ratios
grid = plt.GridSpec(1,2,width_ratios=[5,1]) #rows,columns
ax0, ax1 = [fig.add_subplot(gr) for gr in grid]

# set shared y-axis
ax1.sharey(ax0)

# set ticks on right y-axis
ax0.tick_params('y',right=True)
ax1.tick_params('y',right=True)

# remove left ticks labels from second subplot
ax1.tick_params('y',labelleft=False)

# set title
ax0.set_title(DM.getScan_id(fname))

# set axis labels
if Q:
    ax0.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
ax0.set_ylabel('Time (s)')

if not type(T) == type(None):
    # temperature profile
    ax1.set_xlabel('Temperature (K)')
    ax1.plot(T,t,'r')
else:
    ax1.set_xlabel('I0 (%)')
    ax1.plot(I0*100,t,'r')

ax0.grid(False)
# plot heatmap
ax0.pcolormesh(x,
               t,
               I, 
               shading='nearest',
               norm='log')


## Plot heatmap and correlation matrix <a id='corr'></a>

In [ ]:
# calculate correlation matrix
corr = np.corrcoef(I)

fig, (ax0,ax1) = plt.subplots(1,2)
fig.suptitle(DM.getScan_id(fname))
fig.set_figwidth(10)
ax0.pcolormesh(x,
               t,
               I, 
               shading='nearest',
               norm='log')

ax1.pcolormesh(t,
               t,
               np.abs(corr),
               shading='nearest',
               cmap='viridis')#,vmax=1)

ax1.set_aspect(1)

ax0.set_title('Heatmap')
ax1.set_title('Correlation map')
if Q:
    ax0.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
ax1.set_xlabel('Time (s)')

ax0.set_ylabel('Time (s)')
ax1.set_ylabel('Time (s)')

ax0.grid(False)
ax1.grid(False)

fig.tight_layout()

## Plot heatmap and deviation from mean <a id='mean'></a>

In [ ]:
I[I<=0]=np.nan
# calculate deviation from mean pattern
dev_mean = (I - np.nanmean(I,axis=0))/I*100

fig, (ax0,ax1) = plt.subplots(1,2)
fig.suptitle(DM.getScan_id(fname))
fig.set_figwidth(10)

ax0.pcolormesh(x,
               t,
               I,
               shading='nearest',
               norm='log')

vmax = 5
pcm = ax1.pcolormesh(x,
                     t,
                     dev_mean,
                     shading='nearest',
                     cmap='RdBu_r',
                     vmax=vmax,
                     vmin=-vmax)

ax0.set_title('Heatmap')
ax1.set_title('Deviation from mean')
if Q:
    ax0.set_xlabel('q (A-1)')
    ax1.set_xlabel('q (A-1)')
else:
    ax0.set_xlabel('2theta (deg)')
    ax1.set_xlabel('2theta (deg)')

ax0.set_ylabel('Time (s)')
ax1.set_ylabel('Time (s)')

ax0.grid(False)
ax1.grid(False)

fig.colorbar(pcm,ax=ax1,label='Deviation (%)')

fig.tight_layout()
